In [1]:
from langchain_groq import ChatGroq

In [3]:
llm = ChatGroq(
 temperature = 0,
    groq_api_key = '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',
    model_name = "llama-3.1-70b-versatile"
)

In [5]:
response = llm.invoke("Who is the capitan of indian cricket team in 2011?")
print(response.content)

The captain of the Indian cricket team in 2011 was MS Dhoni.


In [7]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.apple.com/en-us/details/200565688/aiml-machine-learning-platform-infrastructure?team=SFTWR")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.











AIML- Machine Learning Platform & Infrastructure - Careers at Apple







AppleStoreMaciPadiPhoneWatchVisionAirPodsTV & HomeEntertainmentAccessoriesSupport


0+
Careers at AppleOpen MenuClose Menu

      Work at Apple
    
 

      Life at Apple
    
 

      Profile
    
 

      Sign In
    
 
Search
Jobs at Apple
AIML- Machine Learning Platform & InfrastructureSeattle, Washington, United StatesMachine Learning and AIAdd to Favorites AIML- Machine Learning Platform & InfrastructureRemoved from favoritesAdd a favoriteCloseTo view your favorites, sign in with your Apple ID.Sign InDon’t have an Apple ID?Create one nowForgot your Apple ID or password?Submit ResumeAIML- Machine Learning Platform & InfrastructureBack to search resultsSummaryPosted: Sep 4, 2024Role Number:200565688At Apple, great ideas have a way of becoming great products, services, and customer experiences very quickly. Bring passion and dedication to your job and there’s no telling what you could accomplish.

D

In [9]:
from langchain_core.prompts import PromptTemplate

In [19]:
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return the, in JSON format containing
    following keys: `role`, `experience`, `skills` and `description`.
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE)
    """
)

chain_extract = prompt_extract | llm
res = chain_extract.invoke(input={'page_data': page_data})
print(res.content)

[
  {
    "role": "AIML- Machine Learning Platform & Infrastructure",
    "experience": "5+ years of relevant experience",
    "skills": [
      "Strong coding skills in any programming language",
      "Strong problem solving skills and ability to write performant and high quality clean code",
      "Experience solving Big Data problems",
      "Excellent interpersonal skills"
    ],
    "description": "We are responsible for Search infrastructure at Apple. We design and build infrastructure to support features that empowers Spotlight and Siri users. We are responsible for the end to end ownership of our pipelines and services, from conception, to design, to implementation, to deployment, to operations."
  }
]


In [21]:
from langchain_core.output_parsers import JsonOutputParser

In [24]:
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': 'AIML- Machine Learning Platform & Infrastructure',
  'experience': '5+ years of relevant experience',
  'skills': ['Strong coding skills in any programming language',
   'Strong problem solving skills and ability to write performant and high quality clean code',
   'Experience solving Big Data problems',
   'Excellent interpersonal skills'],
  'description': 'We are responsible for Search infrastructure at Apple. We design and build infrastructure to support features that empowers Spotlight and Siri users. We are responsible for the end to end ownership of our pipelines and services, from conception, to design, to implementation, to deployment, to operations.'}]

In [26]:
import pandas as pd

In [28]:
df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [32]:
import chromadb
import uuid

In [34]:
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas = {"links":row["Links"]},
                       ids=[str(uuid.uuid4())])

In [40]:
links = collection.query(query_texts=["Experience in Python", "Experience in React"], n_results=2)
print(links)

{'ids': [['83bc3f16-43f0-4b18-b8bf-6296f2e59c58', '7e71bb26-e701-4c1a-84ce-c3604c7b4a61'], ['91813b04-101f-45bf-9dc6-b484792ced50', 'f42a8d8e-d84a-4f2d-b571-9e032db07324']], 'distances': [[0.9969006380863958, 1.057901867484696], [0.9630503091209697, 1.068565171699208]], 'metadatas': [[{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/python-portfolio'}], [{'links': 'https://example.com/react-portfolio'}, {'links': 'https://example.com/react-native-portfolio'}]], 'embeddings': None, 'documents': [['Machine Learning, Python, TensorFlow', 'Python, Django, MySQL'], ['React, Node.js, MongoDB', 'React Native, Node.js, MongoDB']], 'uris': None, 'data': None, 'included': ['metadatas', 'documents', 'distances']}


In [58]:
job = json_res[0]
print(job['skills'])

['Strong coding skills in any programming language', 'Strong problem solving skills and ability to write performant and high quality clean code', 'Experience solving Big Data problems', 'Excellent interpersonal skills']


In [76]:
links = collection.query(query_texts=job['skills'], n_results=2)
print(links)

metadata = links.get('metadata', [])
print(metadata)

{'ids': [['83bc3f16-43f0-4b18-b8bf-6296f2e59c58', 'c711d072-58aa-4538-938c-0cd3dd966fc0'], ['83bc3f16-43f0-4b18-b8bf-6296f2e59c58', 'c711d072-58aa-4538-938c-0cd3dd966fc0'], ['83bc3f16-43f0-4b18-b8bf-6296f2e59c58', 'b873e2ec-6dc9-4df7-8548-1cf3cd650000'], ['b873e2ec-6dc9-4df7-8548-1cf3cd650000', '83bc3f16-43f0-4b18-b8bf-6296f2e59c58']], 'distances': [[1.3454984913876111, 1.589141790749119], [1.5860821811838788, 1.681917662475185], [1.407658017442574, 1.5975791576545297], [1.7800567539308354, 1.7988076254320264]], 'metadatas': [[{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/magento-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/magento-portfolio'}], [{'links': 'https://example.com/ml-python-portfolio'}, {'links': 'https://example.com/android-portfolio'}], [{'links': 'https://example.com/android-portfolio'}, {'links': 'https://example.com/ml-python-portfolio'}]], 'embeddings': None, 'documents': [[

In [80]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Sri Sai Srivatsa Yerrapragada, I am recently graduated in Master's degree in Electrical and Computer Engineering, specializing in Machine Learning, with a minor in Computer Science at the University of Florida.
        With one year of industry experience as a Software Developer at Accenture, I have developed a profound interest in Machine Learnning and software engineering, demonstrating proficiency in multiple programming languages and databases. 
        Furthermore, I have earned industry-recognized certifications, including the AWS Certified Developer, AWS Solution Architect Associate and AWS Machine Learning Speciality. 
        
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of Sri Sai Srivatsa Yerrapragada 
        in fulfilling their needs.
        
        Also add the most relevant ones from the following links to showcase Srivatsa's portfolio: {link_list}
        Remember you are Sri Sai Srivatsa Yerrapragada. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Application for AIML- Machine Learning Platform & Infrastructure Role at Apple

Dear Hiring Manager,

I am excited to apply for the AIML- Machine Learning Platform & Infrastructure role at Apple, as advertised. With a Master's degree in Electrical and Computer Engineering, specializing in Machine Learning, and a minor in Computer Science from the University of Florida, I am confident that my skills and experience make me an ideal candidate for this position.

As a seasoned software developer with over a year of industry experience at Accenture, I have developed a strong foundation in software engineering and machine learning. My proficiency in multiple programming languages, including Python, Java, and PHP, has enabled me to tackle complex problems and deliver high-quality solutions. Additionally, my experience with various databases has allowed me to effectively manage and analyze large datasets.

I am particularly drawn to this role at Apple because of the opportunity to wor